In [12]:
import numpy as np
import os
import time
import cv2
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import argparse

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras import layers
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from math import *


In [13]:
print(tf.__version__)

2.7.2


In [14]:
!pip list --isolated

Package                       Version
----------------------------- -------------------
absl-py                       0.12.0
aiohttp                       3.8.1
aiosignal                     1.2.0
appdirs                       1.4.4
argon2-cffi                   20.1.0
astropy                       5.0
astunparse                    1.6.3
async-generator               1.10
async-timeout                 4.0.1
attrs                         21.2.0
autoai-libs                   1.13.6
autoai-ts-libs                1.1.9
autovizwidget                 0.18.0
av                            8.0.3
azure-core                    1.21.0
azure-cosmos                  4.2.0
backcall                      0.2.0
beautifulsoup4                4.10.0
biopython                     1.78
black                         19.10b0
bleach                        4.0.0
blinker                       1.4
bokeh                         2.3.3
boto3                         1.18.21
botocore                      1.21.41
Bottl

In [4]:
#!pip uninstall -y tensorflow

In [5]:
#!pip install tensorflow==2.7.2

In [15]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='lBt6MJfLmhKI1KZz5Blf2jrHoLsEbUqnZS1V8WYSE00f',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'effecientdl-donotdelete-pr-bj2mi4n56sagw2'
object_key = 'IBM.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [16]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [17]:
pwd

'/home/wsuser/work'

In [18]:
os.listdir('/home/wsuser/work/IBM')

['TEST_SET', 'TRAIN_SET']

In [19]:
mainPath="/home/wsuser/work/IBM/TRAIN_SET"
destPath="/home/wsuser/work/IBM/DEST"
#mainPath="F:/IBM/TRAIN_SET"
#destPath="F:/work/IBM/DEST"
     
im_size = 224
rangeLower=0
rangeUpper=100
imgsInFile=30
startFileCount=0
augmentCount=5

In [20]:
#Count of fruits
cA=995
cB=1374
cO=1019
cPa=275
cWm=475

#Index of starting ranges
r=[0,cA,cA+cB,cA+cB+cO,cA+cB+cO+cPa]
#Index of ending ranges
er=[cA,cA+cB,cA+cB+cO,cA+cB+cO+cPa,cA+cB+cO+cPa+cWm]

setSize=50

classId=["APPLES","BANANA","ORANGE","PINEAPPLE","WATERMELON"]


In [21]:

inputs = layers.Input(shape=(im_size, im_size, 3))
size = (im_size, im_size)

noOfClasses = 5
imgCategory=np.array([i for i in range(noOfClasses)])
inputImg=[]
imgLabels=[]
    
for i in range(len(classId)):
    dirPath=mainPath+"/"+str(classId[i])
    for j in os.listdir(dirPath): 
        ImgPath=mainPath+"/"+str(classId[i])+"/"+j 
        image = cv2.imread(ImgPath)       
        if(image is not None):
            #BGR->RGB
            img2 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            img3 = cv2.resize(img2, (im_size, im_size))
            img3 = img3.astype('float32') / 255.0       
            inputImg.append(img3)
            imgLabels=imgLabels+[i]
        
    

In [22]:
tr=[r[0],r[1],r[2],r[3],r[4]]


#iteration start
iterationCnt=0
maxItCnt=ceil(max(cA,cB,cO,cPa,cWm)/setSize)

print()
print("Number of Iterations: ",maxItCnt)
print()

print("Model:")
#outputs = EfficientNetB0(include_top=True, weights=None, classes=noOfClasses)(inputs)
outputs=tf.keras.applications.mobilenet.MobileNet(
    include_top=True,
    weights=None,
    classes=noOfClasses
)(inputs)
'''outputs=tf.keras.applications.VGG16(
    include_top=True,
    weights=None,
    classes=noOfClasses
)(inputs)
outputs=tf.keras.applications.inception_v3.InceptionV3(
    include_top=True,
    weights=None,
    classes=noOfClasses
)(inputs)'''
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"] )
model.summary()    


Number of Iterations:  28

Model:
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenet_1.00_224 (Functio  (None, 5)                3233989   
 nal)                                                            
                                                                 
Total params: 3,233,989
Trainable params: 3,212,101
Non-trainable params: 21,888
_________________________________________________________________


In [ ]:
print()
print("Model training:")
print()

while(iterationCnt<maxItCnt):

    tArr=[inputImg[tr[0]:(tr[0]+setSize)%er[0]],inputImg[tr[1]:(tr[1]+setSize)%er[1]],inputImg[tr[2]:(tr[2]+setSize)%er[2]],inputImg[tr[3]:(tr[3]+setSize)%er[3]],inputImg[tr[4]:(tr[4]+setSize)%er[4]]]
    tFinalArr=[]
    tFinalLabels=[]
    for i in range(len(tArr)):
        if(tArr[i]==[]):
            tArr1=inputImg[r[i]:r[i]+1]
            tFinalLabels=tFinalLabels+imgLabels[r[i]:r[i]+1]
        else:
            tFinalLabels=tFinalLabels+imgLabels[tr[i]:tr[i]+setSize]
        tFinalArr=tFinalArr+tArr[i]

    data=np.array(tFinalArr)
    labels=np.array(tFinalLabels)

    df = pd.DataFrame(labels,columns=["Labels"])
    one_hot_encoded_data = pd.get_dummies(df,columns=["Labels"]).to_numpy()

    print()
    print("---------------------------------------------------------------------------------------------------------------------")
    print("Stats for run -",iterationCnt)
    print("---------------------------------------------------------------------------------------------------------------------")
    print("Total no. of img inputs : ",len(inputImg))
    print("Total no. of labels: ",len(imgLabels))
    print("Input data shape: ",data.shape)
    print("Input label shape: ",labels.shape)
    print("Input data size: ",data.size)
    print("Input label size: ",labels.size)
    print("One hot encoded label shape: ",one_hot_encoded_data.shape)
    print("One hot encoded label size: ",one_hot_encoded_data.size)
    print("---------------------------------------------------------------------------------------------------------------------")
    print()


    model.fit(data, one_hot_encoded_data, epochs=10, batch_size=30)
    model.save_weights('weightsV'+str(iterationCnt)+'.h5')
    
    print("---------------------------------------------------------------------------------------------------------------------")
    
    iterationCnt+=1
    
    for i in range(len(tr)):
        if tr[i]+setSize < er[i]:
            tr[i]+=setSize
        else:
            tr[i]=er[i]-1
    
model.save_weights("finalWeights.h5")


Model training:


---------------------------------------------------------------------------------------------------------------------
Stats for run - 0
---------------------------------------------------------------------------------------------------------------------
Total no. of img inputs :  4138
Total no. of labels:  4138
Input data shape:  (250, 224, 224, 3)
Input label shape:  (250,)
Input data size:  37632000
Input label size:  250
One hot encoded label shape:  (250, 5)
One hot encoded label size:  1250
---------------------------------------------------------------------------------------------------------------------

Epoch 1/10
9/9 [==============================] - 37s 4s/step - loss: 1.3521 - accuracy: 0.4320
Epoch 2/10
9/9 [==============================] - 35s 4s/step - loss: 0.7511 - accuracy: 0.6680
Epoch 3/10
9/9 [==============================] - 34s 4s/step - loss: 0.4905 - accuracy: 0.8360
Epoch 4/10
9/9 [==============================] - 32s 4s/step - loss: 0.3

In [3]:
print("Files: ")
!ls -1

Files: 
IBM
weightsV0.h5
weightsV1.h5


In [ ]:
!tar -zcvf modelW1.tgz finalWeights.h5

In [ ]:
!pip install watson-machine-learning-client --upgrade

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey":"LLxjfOIIUfLwAmY9M6K0IAGn7bRmXucM-JgHcxYkkRFZ"
}
client = APIClient(wml_credentials)

In [ ]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])


In [ ]:
space_uid = guid_from_space_name(client,'ImageClassification')
print(space_uid)

In [ ]:
client.set.default_space(space_uid)


In [ ]:
#client.software_specifications.list()

In [ ]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid


In [ ]:
model_details = client.repository.store_model(model='modelW1.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})

In [ ]:
model_id=client.repository.get_model_uid(model_details)
model_id